In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys  import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [78]:
def scroll_all(driver):
    global ready_count
    ready_count = 0
    body = driver.find_element(By.TAG_NAME, "body")
    
    for _ in range(100):  # 10번 페이지 다운
        body.send_keys(Keys.END)
        time.sleep(1)
    
    ready_count = 1


In [79]:
def main():
    global parse
    driver = webdriver.Chrome()

    # login(driver, user_id, passwd)
    scroll_all(driver)       
    # time.sleep(5) 
    # parse = get_page_source(driver)
    
    time.sleep(5)
    driver.close()
    return 

In [80]:
# driver = webdriver.Chrome()

# # login(driver, user_id, passwd)
# driver.get('https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&locations=all')
# # driver.switch_to.frame('mainFrame') # 해당 iframe으로 이동
# scroll_all(driver)

# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

# results = soup.find_all('div', {'class': 'JobCard_JobCard__Tb7pI'})
# # 'span',{'class':'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'}

# # print(results)

# # return results
# print(results)

In [81]:
driver = webdriver.Chrome()

# login(driver, user_id, passwd)
driver.get('https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&locations=all')
# driver.switch_to.frame('mainFrame') # 해당 iframe으로 이동
scroll_all(driver)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

results_a = soup.find_all('div',{'class':'JobCard_JobCard__Tb7pI'})
results0 = soup.find_all('span', {'class': 'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'})
results1 = soup.find_all('span', {'class': 'CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__company__j_pad wds-1m3gvmz'})
results2 = soup.find_all('span', {'class': 'CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__location__FHNmN wds-1m3gvmz'})
# 'span',{'class':'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'}

# print(results)
time.sleep(5)
driver.close()

# return results
print(results_a[0].a['href'])
print(results0[0])
print(results1[0])
print(results2[0])


/wd/267736
<span class="JobCard_JobCard__body__position__CyaGY wds-1mh3acf">소프트웨어 엔지니어</span>
<span class="CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__company__j_pad wds-1m3gvmz">폴라리스오피스</span>
<span class="CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__location__FHNmN wds-1m3gvmz">서울 구로구 · 신입-경력 6년</span>


In [82]:
exportDF = pd.DataFrame(columns=['포지션','회사명','지역명','경력','링크'])

In [83]:
region = []
years = []

In [84]:
for i in range(len(results2)):
    region.append(results2[i].text.split('·')[0])
    years.append(results2[i].text.split('·')[1])

In [85]:
for i in range(len(results0)):
    results0[i] = results0[i].text
    results1[i] = results1[i].text
    results2[i] = results2[i].text
    results_a[i] = results_a[i].a['href']

In [86]:
exportDF['포지션']=results0
exportDF['회사명']=results1
exportDF['지역명']=region
exportDF['경력']=years
exportDF['링크']=results_a

In [87]:
exportDF

,포지션,회사명,지역명,경력,링크
0,소프트웨어 엔지니어,폴라리스오피스,서울 구로구,신입-경력 6년,/wd/267736
1,시니어 개발자 (Rust),베링랩,서울 마포구,경력 5-10년,/wd/206209
2,Unity 개발자,인터엑스,서울 금천구,경력 5-10년,/wd/261266
3,Head of Architecture,인터엑스,서울 금천구,경력 10년 이상,/wd/256936
4,AI Model Optimization Engineer (CPU/GPU/NPU),인터엑스,울산 중구,경력 3-15년,/wd/252960
...,...,...,...,...,...
3214,DBMS 엔진 개발,큐브리드,서울 강남구,경력 1-15년,/wd/118825
3215,Windows Application 개발자,아진엑스텍,경기 군포시,경력 8-12년,/wd/113536
3216,친환경 올인원 에어케어시스템 PCB 및 펌웨어,엔트,경기 부천시,경력 5-10년,/wd/76159
3217,"PCB약품 개발, 제조업, 화학약품",엔알지엔씨,경기 수원시,신입,/wd/87790


In [ ]:
article_list = []
for i in exportDF.index[:5]:
    driver = webdriver.Chrome()
    driver.get(f"{'https://www.wanted.co.kr' + exportDF.loc[i,'링크']}")
    try:
        # "상세정보 더 보기" 버튼 찾기 (버튼의 텍스트가 '상세정보 더 보기'일 경우)
        button = driver.find_element(By.XPATH, "//*[@id='__next']/main/div[1]/div/section/section/article[1]/div/button")
        
        # 버튼 클릭
        button.click()
        
        # 클릭 후 페이지가 갱신될 시간을 기다림
        time.sleep(1)
    except :
        pass
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    article = soup.find('article',{'class':"JobDescription_JobDescription__dq8G5"})
    article_list.append(article.text)
   
    
    time.sleep(1)
    driver.close()

In [97]:
article_list

["포지션 상세폴라리스오피스에서 새로운 성장을 함께 만들어 나갈 인재를 모십니다.!폴라리스오피스는 단 하나의 앱으로 워드, 엑셀, 파워포인트 등 모든 포맷의 문서를 언제 어디서든 편집할 수 있는 글로벌 오피스 소프트웨어입니다. 모바일, PC 다양한 환경에서 최적화된 서비스를 제공하고  있으며, 안정적이고 뛰어난 호환성으로 원격근무와 스마트오피스 필수 앱으로 자리매김 하였습니다.매일 평균 5만 명의 신규 가입자가 발생하고 현재까지 1.3억명 이상의 글로벌 유저와 60여개 글로벌 고객사를 지니고 있으며, 국내 최초 생성형 언어모델을 탑재한 오피스AI 솔루션 'Nova'를 출시하여 오피스시장을 선도하고 있습니다. 가파른 성장세로 2024년, 폴라리스AI와 폴라리스AI파마를 인수하여 5개 코스닥 상장사로 이루어진 '폴라리스오피스그룹'의 모회사가 되었으며 소프트웨어 뿐만 아니라 다양한 비즈니스에서 AI융합기술 시너지를 모색하고 있습니다. [팀 소개]기업개발팀은 Polaris Office Engine기반의 B2B 서비스 Polaris Docs와 Polaris Converter 개발 및 유지 보수를 합니다.Polaris Docs는 문서 중앙화 기반의 공동편집 솔루션으로써 문서 협업에 최적화된 환경을 제공하기 위해 고객사 시스템환경과 요구사항에 맞게 개발을 하고, 리눅스 컨테이너 환경에서 솔루션을 구축하는 경험을 할 수 있습니다. Polaris Converter는 다양한 문서에 대해서 PDF 또는 이미지 형태로 변환하는 문서 변환 솔루션으로, 완제품 형태의 서버형과 기존 서비스에 연동하여 사용하는 Libary형 제품으로 제공되며, 변화 하는 시장에 맞춰 꾸준한 업그레이드가 진행 됩니다.본인이 가지고 있는 아이디어와 새로운 기술을 적용하는데에 있어 적극적으로 지원하고 있으니 관심있는 분들의 지원을 기다리겠습니다.주요업무• Docker 기반 오피스 솔루션 구축 및 유지보수• 기업 요청사항에 따른 솔루션 기능개발[사용하는 기술 및 환경]• Docker, Spring Java, J